In [69]:
import pandas as pd 


In [71]:
sentences = pd.read_csv("../text/full.csv")
sentences.head()

,theme,type,structure,numer,tense,polarity,sentence,num_words
0,weather,imperative,cleft,singular,present,negative,Don’t forget that it rains in spring\n,7
1,weather,imperative,cleft,singular,present,negative,Please remember that winter is not always cold\n,8
2,weather,imperative,cleft,singular,present,negative,It is not my fault when the thunder strikes\n,9
3,weather,imperative,cleft,singular,present,negative,You should know that summer does not last fore...,9
4,weather,imperative,cleft,singular,present,negative,Do not believe that snow is never beautiful\n,8


In [72]:
short_sents = sentences.query("num_words <= 8")

short_sents[['theme', 'type', 'structure']].value_counts()

theme    type         structure  
food     exclamatory  simple         238
basic    exclamatory  simple         237
emotion  exclamatory  simple         230
basic    declarative  simple         230
health   exclamatory  simple         228
                                    ... 
nature   declarative  independent      2
art      declarative  independent      2
food     imperative   independent      2
art      imperative   independent      1
housing  imperative   independent      1
Name: count, Length: 300, dtype: int64

In [73]:
short_sents.structure.value_counts()

structure
simple         10058
preposition     4494
cleft           3643
dependent       2341
independent      903
Name: count, dtype: int64

In [74]:
short_sents.type.value_counts()

type
exclamatory      7777
interrogative    4842
declarative      4771
imperative       4049
Name: count, dtype: int64

In [75]:
short_sents.polarity.value_counts()

polarity
affirmative    12189
negative        9250
Name: count, dtype: int64

current strategy: 
- theme: don't care 
- type: declarative
- structure: simple 
- tense: present 
- polarity: positive 

In [96]:
max_words = 8
structure = 'simple'
tense = 'present'
polarity = 'affirmative'

subset = sentences.query(f"num_words <= {max_words} and structure == '{structure}' and tense == '{tense}' and polarity == '{polarity}' and type == 'declarative'")
subset.head()


,theme,type,structure,numer,tense,polarity,sentence,num_words
460,food,declarative,simple,singular,present,affirmative,I enjoy eating fresh fruits every morning\n,7
461,food,declarative,simple,singular,present,affirmative,She drinks water during lunch every day\n,7
462,food,declarative,simple,singular,present,affirmative,He likes to bake cookies on weekends\n,7
463,food,declarative,simple,singular,present,affirmative,My brother loves trying new recipes regularly\n,7
464,food,declarative,simple,singular,present,affirmative,I prefer tea over coffee in the afternoon\n,8


for each sentence type, select 25 sentences 

In [97]:
selected = pd.DataFrame()
total = 256

for i, group in subset.groupby('type'):
    selected = pd.concat([selected, group.sample(total)], ignore_index=True)
    

In [98]:
selected.groupby(['type', 'theme']).sentence.count()

type         theme       
declarative  art             14
             basic           23
             emotion         23
             food            13
             health          23
             housing         21
             humanity        12
             nature          20
             relationship    11
             science         14
             shopping        15
             sport           16
             transport       18
             weather         14
             work            19
Name: sentence, dtype: int64

In [101]:
# save to csv
selected.to_csv('../text/selected.csv', index=False)

In [105]:
translated = pd.read_csv("../text/translation.csv")

assert len(selected) == len(translated)

In [106]:
len(translated)

256

In [107]:
translated.head()

,phrase
0,L'énergie circule constamment d'un système à l...
1,Les diplômés célèbrent leurs réalisations avec...
2,Une grenouille saute gracieusement dans l'étang\n
3,Les joueurs encouragent leurs équipes préférées\n
4,Les programmeurs écrivent du code pour créer d...


In [108]:
selected['translation'] = translated.phrase

In [109]:
def count_words(s):
    remove = ['.', '?', '!', '-', ',']
    cleaned = "".join([c for c in s if c not in remove])
    
    return len(cleaned.split())
        

In [110]:
selected['translanted_num_words'] = selected.translation.apply(count_words)
selected['translanted_num_words'].value_counts()

translanted_num_words
9     64
10    53
8     48
7     30
11    27
6     12
12     8
13     5
5      4
4      3
14     2
Name: count, dtype: int64

In [111]:
selected.head()

,theme,type,structure,numer,tense,polarity,sentence,num_words,translation,translanted_num_words
0,science,declarative,simple,singular,present,affirmative,Energy flows from one system to another consta...,8,L'énergie circule constamment d'un système à l...,7
1,work,declarative,simple,plural,present,affirmative,Graduates celebrate their achievements with fa...,8,Les diplômés célèbrent leurs réalisations avec...,11
2,nature,declarative,simple,singular,present,affirmative,A frog jumps gracefully into the pond\n,7,Une grenouille saute gracieusement dans l'étang\n,6
3,basic,declarative,simple,plural,present,affirmative,Players cheer for their favorite teams\n,6,Les joueurs encouragent leurs équipes préférées\n,6
4,science,declarative,simple,plural,present,affirmative,Programmers write code to build interactive so...,8,Les programmeurs écrivent du code pour créer d...,11


In [112]:
selected.to_csv('../text/selected_with_translation.csv', index=False)